<a href="https://colab.research.google.com/github/keysenelvial/dinam/blob/main/dinam6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Лабораторная работа №6**

Провести классификацию найденного датасета, методами CatBoost.

In [2]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 2.7 MB/s eta 0:00:00


In [4]:
import numpy as np
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV


data = pd.read_csv('/content/drive/MyDrive/train 3.csv', on_bad_lines='skip')
print(data.head())


data = data[['log_price', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'review_scores_rating', 'property_type']].dropna()

# Уменьшение размера датасета
sampled_data = data.sample(frac=0.15, random_state=42)
sampled_data['price_category'] = pd.qcut(sampled_data['log_price'], q=4, labels=False)

# Подготовка признаков и целевой переменной
X = sampled_data[['accommodates', 'bathrooms', 'bedrooms', 'beds']]
y = sampled_data['price_category']

# Разделение данных
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Подготовка модели CatBoost и GridSearchCV
model = CatBoostClassifier(
    iterations=100,
    learning_rate=0.1,
    depth=6,
    silent=True,
    task_type="CPU",
    random_state=42
)


params = {
    'depth': [4, 6, 8],
    'learning_rate': [0.01, 0.1, 0.2],
    'iterations': [50, 100, 200]
}

grid_search = GridSearchCV(model, param_grid=params, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

print('Лучшие параметры:', grid_search.best_params_)

# Обучение модели с лучшими параметрами
best_model = grid_search.best_estimator_
best_model.fit(X_train, y_train)

y_pred = best_model.predict(X_test)

print(classification_report(y_test, y_pred))
print("Границы категорий цен:", pd.qcut(data['log_price'], q=4).unique())


         id  log_price property_type        room_type  \
0   6901257   5.010635     Apartment  Entire home/apt   
1   6304928   5.129899     Apartment  Entire home/apt   
2   7919400   4.976734     Apartment  Entire home/apt   
3  13418779   6.620073         House  Entire home/apt   
4   3808709   4.744932     Apartment  Entire home/apt   

                                           amenities  accommodates  bathrooms  \
0  {"Wireless Internet","Air conditioning",Kitche...             3        1.0   
1  {"Wireless Internet","Air conditioning",Kitche...             7        1.0   
2  {TV,"Cable TV","Wireless Internet","Air condit...             5        1.0   
3  {TV,"Cable TV",Internet,"Wireless Internet",Ki...             4        1.0   
4  {TV,Internet,"Wireless Internet","Air conditio...             2        1.0   

   bed_type cancellation_policy  cleaning_fee  ...   latitude   longitude  \
0  Real Bed              strict          True  ...  40.696524  -73.991617   
1  Real Bed     